In [7]:
import csv
import pandas as pd
import numpy as np
import string

spam_or_ham = pd.read_csv("datasets/svc-spam.csv", encoding='latin-1', on_bad_lines='skip')[["v1", "v2"]]
spam_or_ham.columns = ["label", "text"]
spam_or_ham.head()

print (spam_or_ham["label"].value_counts())

punctuation = set(string.punctuation)

def tokenize(sentence):
    tokens = []
    for token in sentence.split():
        new_token = []
        for character in token:
            if character not in punctuation:
                new_token.append(character.lower())
        if new_token:
            tokens.append("".join(new_token))
    return tokens

spam_or_ham.head()["text"].apply(tokenize)

from sklearn.feature_extraction.text import CountVectorizer
demo_vectorizer = CountVectorizer(
    tokenizer = tokenize,
    binary = True
)

from sklearn.model_selection import train_test_split
train_text, test_text, train_labels, test_labels = train_test_split(spam_or_ham["text"], spam_or_ham["label"], stratify=spam_or_ham["label"])

print(f"Training examples: {len(train_text)}, testing examples {len(test_text)}")

real_vectorizer = CountVectorizer(tokenizer = tokenize, binary=True, token_pattern=None)
train_X = real_vectorizer.fit_transform(train_text)
test_X  = real_vectorizer.transform(test_text)

from sklearn.svm import LinearSVC
classifier = LinearSVC()

classifier.fit(train_X, train_labels)
LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

from sklearn.metrics import accuracy_score

predicciones = classifier.predict(test_X)
accuracy = accuracy_score(test_labels, predicciones)

print(f"Accuracy: {accuracy:.4%}\n")

frases = [
  'Are you looking to redesign your website with new modern look and feel?',
  'Please send me a confirmation of complete and permanent erasure of the personal data',
  'You have been selected to win a FREE suscription to our service',
  'We’re contacting you because the webhook endpoint associated with your account in test mode has been failing'
  ]

frases_X = real_vectorizer.transform(frases)
predicciones = classifier.predict(frases_X)

for text, label in zip(frases, predicciones):
    print(f"{label:5} -{text}")

ham     4780
spam     742
Name: label, dtype: int64
Training examples: 4141, testing examples 1381
Accuracy: 98.4070%

spam  -Are you looking to redesign your website with new modern look and feel?
ham   -Please send me a confirmation of complete and permanent erasure of the personal data
spam  -You have been selected to win a FREE suscription to our service
ham   -We’re contacting you because the webhook endpoint associated with your account in test mode has been failing
